<center><h1>DSCI-552 HOMEWORK 7</h1><center>
<br>
<center><font size="4"></font></center>
<center><font size="3"><strong>Mason(Mohan) Xing</font></center>
<center><font size="3"><strong>USCID:	6880083372</font></center>

In [1]:
# import dependencies
# import warnings
# warnings.filterwarnings("ignore")

import os
import glob
import re
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
# check if GPU is mounted
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow version:", tf.__version__)


# check the environment
print (os.environ['CONDA_DEFAULT_ENV'])

['/device:CPU:0', '/device:GPU:0']
Num GPUs Available:  1
TensorFlow version: 2.7.0
dsci552


# 1. Generative Models for Text

## (a) In this problem, we are trying to build a generative model to mimic the writing style of prominent British Mathematician, Philosopher, prolific writer, and political activist, Bertrand Russell.
## (b) Download the following books from Project Gutenberg 
http://www.gutenberg.org/ebooks/author/355 in text format:
## (c) LSTM: Train an LSTM to mimic Russell's style and thoughts:
### 1(c) i. Concatenate your text files to create a corpus of Russell's writings.

In [2]:
# test if all txt file can be read
for root, _, files in os.walk('../data/Book Files/books/'):
    file_names_full = files
    for file in files:
        try:
            with open(root + file, 'r') as f:
                f.read()
        except:
            error_file = file
            print("error file is:", error_file)
print("Used data files are: ", file_names_full)

error file is: OKEWFSMP.txt
Used data files are:  ['AIIMAT.txt', 'MLOE.txt', 'OKEWFSMP.txt', 'TAM.txt', 'TAMatter.txt', 'THWP.txt', 'TPP.txt']


In [3]:
# read txt file based on different encoding method
file_names = [x.rstrip('.txt') for x in file_names_full]
book={}
for root, _, files in os.walk('../data/Book Files/books/'):
    for file in files:
        if file == error_file:
            with open(root + file, 'r', encoding="utf8") as f:
                book[file.rstrip('.txt')]=f.readlines()
        else:
            with open(root + file, 'r') as f:
                book[file.rstrip('.txt')]=f.readlines()

In [4]:
# original data sample display
book['AIIMAT'][:18]

['ï»¿INTRODUCTION \n',
 '\n',
 '\n',
 '\n',
 'The present work is intended as an investigation of certain \n',
 'problems concerning empirical knowledge. As opposed to tradi- \n',
 'tional theory of knowledge, the method adopted differs chiefly \n',
 'in the importance attached to linguistic considerations â€¢ I propose \n',
 'to consider language in relation to two main problems, which, in \n',
 'preliminary and not very precise terms, may be stated as follows : \n',
 '\n',
 '. What is meant by "empirical evidence for the truth of a \n',
 '\n',
 'proposition" \n',
 '\n',
 'II. What can be inferred from the fact that there sometimes is \n',
 '\n',
 'such evidence .^ \n']

In [5]:
# function to count upper case ratio each line
def upper_count(str_obj):
    upper_count = len(re.findall(r'[A-Z]',str_obj))
    total_count = len(str_obj)
    ratio = upper_count/total_count
    return(ratio)

In [6]:
# clean up txt
corpus = ""
for file_name in file_names:
    temp_list = []
    for line in book[file_name]:
        # remove "\n"
        line1 = line.replace("- \n", "")
        line2 = line1.replace("\n", " ")
        # remove non alphaneumaric characters
        line3 = re.sub('[^a-zA-Z\.\,\!\?]',' ', line2)
        # remove numeric values
        line4 = re.sub("\d+", "", line3)
        # remove title
        line_uppercount = upper_count(line4)
        if line_uppercount < 0.7:
            corpus+=line4

In [7]:
# remove extra white space
corpus_clean =" ".join(corpus.split())
corpus_clean[:500]

'The present work is intended as an investigation of certain problems concerning empirical knowledge. As opposed to traditional theory of knowledge, the method adopted differs chiefly in the importance attached to linguistic considerations I propose to consider language in relation to two main problems, which, in preliminary and not very precise terms, may be stated as follows . What is meant by empirical evidence for the truth of a proposition II. What can be inferred from the fact that there so'

### 1(c) ii. 
Use a character-level representation for this model by using extended ASCII that has N = 256 characters. 
Each character will be encoded into a an integer using its ASCII code. Rescale the integers to the range [0; 1], because LSTM uses a sigmoid activation function. LSTM will receive the rescaled integers as its input

In [8]:
# define a function to convert character to ASCII code
def to_ascii(text):
    ascii_values = [ord(character) for character in text]
    return ascii_values

In [9]:
scaler = MinMaxScaler()
# encoded into a an integer using its ASCII code
corpus_lower = corpus_clean.lower()
char_ascii = to_ascii(corpus_lower)
ascii_array = np.array(char_ascii).reshape(-1,1)
fitted_scaler = scaler.fit(ascii_array)

In [10]:
# define a function to scale ASCII encoded characters
def rescale(scaler, ascii_4trans):
    '''
    scaler: MinMaxScaler() after fitting
    ascii_4trans : a list of ascii chars used for transforming
    return : scaled ascii array
    
    '''
    ascii_array_4trans = np.array(ascii_4trans).reshape(-1,1)
    scale_ascii = scaler.transform(ascii_array_4trans)
    
    return scale_ascii

### 1(c) iii. & 1(c) iv.
Choose a window size, e.g., W = 100.<br>
Inputs to the network will be the first W-1 = 99 characters of each sequence, and the output of the network will be the Wth character of the sequence.

In [11]:
# define a function to generate X_train, y_train
def X_y_prepare(w, scaler, trans_ascii):
    
    X_list = []
    y_list = []
    scale_char_ascii = rescale(fitted_scaler, trans_ascii)
    for i in tqdm(range(len(scale_char_ascii)-w+1)):
        
        x_i = scale_char_ascii[i:i+w-1]
        y_i = trans_ascii[i+w-1]
        X_list.append(x_i)
        y_list.append(y_i)
        
    X = np.array(X_list).reshape(len(X_list), w-1, 1)
    y = np.array(y_list)
    
    return X, y

In [12]:
w = 100
scaler = MinMaxScaler()
# encoded into a an integer using its ASCII code
corpus_lower = corpus_clean.lower()
char_ascii = to_ascii(corpus_lower)
X, y = X_y_prepare(w, scaler, char_ascii)

print(X.shape)
print(y.shape)

100%|██████████| 4919620/4919620 [00:02<00:00, 1774569.96it/s]


(4919620, 99, 1)
(4919620,)


### 1(c) v. 
Note that the output has to be encoded using a one-hot encoding scheme with N = 256 (or less) elements. This means that the network reads integers, but outputs a vector of N = 256 (or less) elements.

In [13]:
y_encode = to_categorical(y)
print(y_encode.shape)

(4919620, 123)


### 1(c) vi & 1(c) vii. 
Use a single hidden layer for the LSTM with N = 256 (or less) memory units.<br>
Use a Softmax output layer to yield a probability prediction for each of the characters between 0 and 1.<br>
ref: https://www.analyticsvidhya.com/blog/2021/08/predict-the-next-word-of-your-text-using-long-short-term-memory-lstm/<br>
ref: https://towardsdatascience.com/lstm-how-to-train-neural-networks-to-write-like-lovecraft-e56e1165f514<br>

In [14]:
LSTM_model = Sequential()
LSTM_model.add(LSTM(256, input_shape=(X.shape[1], 1)))
LSTM_model.add(Dropout(0.5))
LSTM_model.add(Dense(y_encode.shape[1], activation='softmax'))
LSTM_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 123)               31611     
                                                                 
Total params: 295,803
Trainable params: 295,803
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
LSTM_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy','accuracy'])

### 1(c) viii & 1(c) ix & 1(c) x 
We do not use a test dataset. We are using the whole training dataset to learn the probability of each character in a sequence. We are not seeking for a very accurate model. Instead we are interested in a generalization of the dataset that can mimic the gist of the text..<br>
Choose a reasonable number of epochs for training, considering your computational power (e.g., 30, although the network will need more epochs to yield a better model).<br>
Use model checkpointing to keep the network weights to determine each time an improvement in loss is observed at the end of the epoch. Find the best set of weights in terms of loss.
ref: https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network<br>

In [16]:
file_path = "../model/LSTM/dropout_0.5/"
for f in os.listdir(file_path):
    os.remove(os.path.join(file_path, f))

model_file=file_path + "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(model_file, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [17]:
# Fit the model to the training data
LSTM_model.fit(X, y_encode, epochs=30 , batch_size=128, callbacks=callbacks_list)

Epoch 1/30
38434/38435 [============================>.] - ETA: 0s - loss: 2.5152 - categorical_accuracy: 0.2711 - accuracy: 0.2711
Epoch 00001: loss improved from inf to 2.51522, saving model to ../model/LSTM/dropout_0.5\weights-improvement-01-2.5152.hdf5
38435/38435 [==============================] - 335s 9ms/step - loss: 2.5152 - categorical_accuracy: 0.2711 - accuracy: 0.2711
Epoch 2/30
38434/38435 [============================>.] - ETA: 0s - loss: 2.1633 - categorical_accuracy: 0.3705 - accuracy: 0.3705
Epoch 00002: loss improved from 2.51522 to 2.16330, saving model to ../model/LSTM/dropout_0.5\weights-improvement-02-2.1633.hdf5
38435/38435 [==============================] - 331s 9ms/step - loss: 2.1633 - categorical_accuracy: 0.3705 - accuracy: 0.3705
Epoch 3/30
38433/38435 [============================>.] - ETA: 0s - loss: 2.0366 - categorical_accuracy: 0.4080 - accuracy: 0.4080
Epoch 00003: loss improved from 2.16330 to 2.03661, saving model to ../model/LSTM/dropout_0.5\weights

38435/38435 [==============================] - ETA: 0s - loss: 1.7226 - categorical_accuracy: 0.4991 - accuracy: 0.4991
Epoch 00022: loss improved from 1.72746 to 1.72264, saving model to ../model/LSTM/dropout_0.5\weights-improvement-22-1.7226.hdf5
38435/38435 [==============================] - 324s 8ms/step - loss: 1.7226 - categorical_accuracy: 0.4991 - accuracy: 0.4991
Epoch 23/30
38433/38435 [============================>.] - ETA: 0s - loss: 1.7180 - categorical_accuracy: 0.5003 - accuracy: 0.5003
Epoch 00023: loss improved from 1.72264 to 1.71803, saving model to ../model/LSTM/dropout_0.5\weights-improvement-23-1.7180.hdf5
38435/38435 [==============================] - 324s 8ms/step - loss: 1.7180 - categorical_accuracy: 0.5003 - accuracy: 0.5003
Epoch 24/30
38434/38435 [============================>.] - ETA: 0s - loss: 1.7138 - categorical_accuracy: 0.5015 - accuracy: 0.5015
Epoch 00024: loss improved from 1.71803 to 1.71377, saving model to ../model/LSTM/dropout_0.5\weights-impr

### 1(c)xi. Use the network with the best weights to generate 1000 characters

In [18]:
def generate_newtext(model, text, w, scale):
    
    for i in tqdm(range(1000)):
    
        text_lower = text.lower()
        text_ascii = to_ascii(text_lower)
        seq = text_ascii[-(w-1):]
        seq_scale = rescale(fitted_scaler, seq)
        seq_scale_arr = np.array(seq_scale)
        X_test =  seq_scale_arr.reshape(1,w-1,1)
        y_pred = model.predict(X_test)
        y_pred_ascii = np.argmax(y_pred, axis=None, out=None)
        y_pred_char = chr(y_pred_ascii)
        text+=y_pred_char
    
    return text

In [19]:
original_test_text = "There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object. "

In [20]:
# loading the saved model
folder_path = file_path
file_type = '\*.hdf5'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)

loaded_model = load_model(max_file)
test_text = original_test_text + ''
w = 100

new_text = generate_newtext(loaded_model, test_text, w, fitted_scaler)
print(new_text)

100%|██████████| 1000/1000 [00:33<00:00, 29.80it/s]

There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object. the soatate seannd seannd aelieting the soace of the soace of the soace of the soace of the soace of the soace thmng that iave the semse data of the soace thmng that iave the semse data and the soace of the soace thmng that iave the semse data and the soace of the soace thmng that iave the semse data and the soace of the soace thmng that iave the semse data and the soace of the soace thmng that iave the semse data and the soace of the soace thmng that iave the semse data and the soace of the soace thmng that iave the semse data and the soace of the soace thmng that iave the semse data and the soace of the soace thmng that iave the semse data and the soace of the soace thmng that iave the semse data and the soace of the soace thmng that iave the semse data and the soace of the soace thmng that iave the semse data and the soace of the s

In [21]:
# fit the model for 30 epoch more
loaded_model.fit(X, y_encode, epochs=30, batch_size=128, callbacks=callbacks_list)

Epoch 1/30
38433/38435 [============================>.] - ETA: 0s - loss: 1.6880 - categorical_accuracy: 0.5089 - accuracy: 0.5089
Epoch 00001: loss improved from 1.69128 to 1.68796, saving model to ../model/LSTM/dropout_0.5\weights-improvement-01-1.6880.hdf5
38435/38435 [==============================] - 326s 8ms/step - loss: 1.6880 - categorical_accuracy: 0.5089 - accuracy: 0.5089
Epoch 2/30
38432/38435 [============================>.] - ETA: 0s - loss: 1.6837 - categorical_accuracy: 0.5098 - accuracy: 0.5098
Epoch 00002: loss improved from 1.68796 to 1.68366, saving model to ../model/LSTM/dropout_0.5\weights-improvement-02-1.6837.hdf5
38435/38435 [==============================] - 325s 8ms/step - loss: 1.6837 - categorical_accuracy: 0.5098 - accuracy: 0.5098
Epoch 3/30
38431/38435 [============================>.] - ETA: 0s - loss: 1.7795 - categorical_accuracy: 0.4844 - accuracy: 0.4844
Epoch 00003: loss did not improve from 1.68366
38435/38435 [==============================] - 326

38431/38435 [============================>.] - ETA: 0s - loss: nan - categorical_accuracy: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 00027: loss did not improve from 1.68366
38435/38435 [==============================] - 325s 8ms/step - loss: nan - categorical_accuracy: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 28/30
38432/38435 [============================>.] - ETA: 0s - loss: nan - categorical_accuracy: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 00028: loss did not improve from 1.68366
38435/38435 [==============================] - 325s 8ms/step - loss: nan - categorical_accuracy: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 29/30
38433/38435 [============================>.] - ETA: 0s - loss: nan - categorical_accuracy: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 00029: loss did not improve from 1.68366
38435/38435 [==============================] - 325s 8ms/step - loss: nan - categorical_accuracy: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 30/30
38434/38435 [============================>.] - ETA: 0s 

In [22]:
# loading the saved model
folder_path = file_path
file_type = '\*.hdf5'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)

loaded_model = load_model(max_file)
test_text = original_test_text + ''
w = 100

new_text = generate_newtext(loaded_model, test_text, w, fitted_scaler)
print(new_text)

100%|██████████| 1000/1000 [00:33<00:00, 29.96it/s]

There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object. the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of the soace of t